In [ ]:
# RAG 파이프라인
# !pip install langchain_chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_spliter.split_documents(documents)

embedding = OpenAIEmbeddings(model='text-embedding-3-small')
db = Chroma.from_documents(split_documents, embedding)

prompt = ChatPromptTemplate.from_template(
    '''
    다음 문맥만을 고려해서 질문에 답하시요.


    문맥: """{context}"""
   
    질문: {question}
    '''
)

retriever = db.as_retriever()
chain = {
    "question" : RunnablePassthrough(),
    "context" : retriever,
} | prompt | model | StrOutputParser()

output = chain.invoke("LangChain 개요를 알려줘?")

In [ ]:
# HyDE RAG
hypothetical_prompt = ChatPromptTemplate.from_template("""
다음 질문에 한 문장으로 답할 것!
질문 : {question}
""")
hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

hype_rag_chain = {
    "question" : RunnablePassthrough(),
    "context" : hypothetical_chain |  retriever,
} | prompt | model | StrOutputParser()

In [ ]:
#  Multi-Query RAG
from pydantic import BaseModel, Field
class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(description='검색 쿼리 목록')
query_generation_prompt = ChatPromptTemplate.from_template("""
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하기 위한 3개의 서로 다른 검색 쿼리를 생성하세요.
거리 기반 유사성 검색의 한계를 극복하기 위해 사용자의 질문에 대해서 여러 관점을 제공하는 것이 목표입니다.
질문: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x : x.queries)
)

multi_query_rag_chain = {   "question" : RunnablePassthrough(),
    'context': query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

output = multi_query_rag_chain.invoke("LangChain 개요를 알려줘?")
pprint.pprint(output)